# From structure to Graph


Este notebook es la mínima funcionalidad para convertir data estructurada + ontología en grafo de conocimiento.

In [1]:
# PROCESAMIENTO DE DATOS
# 
from langchain_experimental.graph_transformers import LLMGraphTransformer

# AGENTES Y ORQUESTACIÓN
from langchain_ollama import ChatOllama, OllamaEmbeddings
from typing import List
from langgraph.graph import END, START, StateGraph


from IPython.display import Image, display, Markdown
import pandas as pd

In [2]:
def add_src_to_path():
    '''
    '''
    import sys
    import importlib.util
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / "src"

        if candidate.exists():
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
                print(f"Added {candidate} to sys.path")
            else:
                print(f"{candidate} already in sys.path")
            break
        else:
            print(f"{candidate} does not exist")


In [3]:
add_src_to_path()

C:\Users\alejandro.gl\Documents\_projects\Onto_Research\notebooks\src does not exist
Added C:\Users\alejandro.gl\Documents\_projects\Onto_Research\src to sys.path


### Cargar data

In [10]:
df = pd.read_csv("../src/db/structured/supply_chain_data.csv")
df

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,haircare,SKU95,77.903927,65,672,7386.363944,Unknown,15,14,26,...,Mumbai,18,450,26,58.890686,Pending,1.210882,Air,Route A,778.864241
96,cosmetics,SKU96,24.423131,29,324,7698.424766,Non-binary,67,2,32,...,Mumbai,28,648,28,17.803756,Pending,3.872048,Road,Route A,188.742141
97,haircare,SKU97,3.526111,56,62,4370.916580,Male,46,19,4,...,Mumbai,10,535,13,65.765156,Fail,3.376238,Road,Route A,540.132423
98,skincare,SKU98,19.754605,43,913,8525.952560,Female,53,1,27,...,Chennai,28,581,9,5.604691,Pending,2.908122,Rail,Route A,882.198864


In [11]:
df.columns

Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Stock levels', 'Lead times', 'Order quantities', 'Shipping times',
       'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location',
       'Lead time', 'Production volumes', 'Manufacturing lead time',
       'Manufacturing costs', 'Inspection results', 'Defect rates',
       'Transportation modes', 'Routes', 'Costs'],
      dtype='object')

## Model graph



In [ ]:
query = """ 

// Crear nodos para cada Producto (SKU)
CREATE (p:Product {sku: 'SKU0', product_type: 'haircare'});
CREATE (p:Product {sku: 'SKU1', product_type: 'skincare'});
CREATE (p:Product {sku: 'SKU2', product_type: 'cosmetics'});

// Crear nodos para los Proveedores
CREATE (s:Supplier {name: 'Supplier 3'});
CREATE (s:Supplier {name: 'Supplier 1'});
CREATE (s:Supplier {name: 'Supplier 5'});

// Crear nodos para las Ubicaciones
CREATE (l:Location {city: 'Mumbai'});
CREATE (l:Location {city: 'Kolkata'});
CREATE (l:Location {city: 'Delhi'});

// Crear nodos para los Transportistas
CREATE (sc:ShippingCarrier {name: 'Carrier B'});
CREATE (sc:ShippingCarrier {name: 'Carrier A'});
CREATE (sc:ShippingCarrier {name: 'Carrier C'});

// Crear nodos para las Rutas de Transporte
CREATE (r:Route {name: 'Route B'});
CREATE (r:Route {name: 'Route C'});
CREATE (r:Route {name: 'Route A'});

// Crear nodos para los Modos de Transporte
CREATE (tm:TransportationMode {mode: 'Road'});
CREATE (tm:TransportationMode {mode: 'Air'});
CREATE (tm:TransportationMode {mode: 'Rail'});


// Crear nodos para representar la información de Ventas de cada producto
CREATE (s:Sale {
  number_of_products_sold: 802,
  revenue_generated: 8661.996792392383,
  customer_demographics: 'Non-binary'
});

CREATE (s:Sale {
  number_of_products_sold: 736,
  revenue_generated: 7460.900065445849,
  customer_demographics: 'Female'
});

CREATE (s:Sale {
  number_of_products_sold: 8,
  revenue_generated: 9577.74962586873,
  customer_demographics: 'Unknown'
});

// Crear nodos para el Inventario
CREATE (inv:Inventory {
  stock_levels: 58,
  availability: 55
});

CREATE (inv:Inventory {
  stock_levels: 53,
  availability: 95
});

CREATE (inv:Inventory {
  stock_levels: 1,
  availability: 34
});

// Crear nodos para la Producción
CREATE (prod:Production {
  production_volumes: 215,
  manufacturing_lead_time: 29,
  manufacturing_costs: 46.27987924050832
});

CREATE (prod:Production {
  production_volumes: 517,
  manufacturing_lead_time: 30,
  manufacturing_costs: 33.61676895373
});

CREATE (prod:Production {
  production_volumes: 971,
  manufacturing_lead_time: 27,
  manufacturing_costs: 30.688019348284204
});

// Crear nodos para la Calidad
CREATE (qc:QualityCheck {
  inspection_results: 'Pending',
  defect_rates: 0.2264103608499251
});

CREATE (qc:QualityCheck {
  inspection_results: 'Fail',
  defect_rates: 4.854068026388706
});

CREATE (qc:QualityCheck {
  inspection_results: 'Pass',
  defect_rates: 4.580592619199229
});

// Crear nodos para el Envío
CREATE (ship:Shipment {
  order_quantities: 96,
  shipping_times: 4,
  shipping_costs: 2.956572139430807
});

CREATE (ship:Shipment {
  order_quantities: 37,
  shipping_times: 2,
  shipping_costs: 9.71657477143131
});

CREATE (ship:Shipment {
  order_quantities: 88,
  shipping_times: 6,
  shipping_costs: 8.054479261732155
});


////
// Conectar Productos con su información de Ventas e Inventario
MATCH (p:Product {sku: 'SKU0'}), (s:Sale {number_of_products_sold: 802}), (inv:Inventory {stock_levels: 58})
CREATE (p)-[:HAS_SALE_INFO]->(s)
CREATE (p)-[:HAS_INVENTORY_INFO]->(inv);

MATCH (p:Product {sku: 'SKU1'}), (s:Sale {number_of_products_sold: 736}), (inv:Inventory {stock_levels: 53})
CREATE (p)-[:HAS_SALE_INFO]->(s)
CREATE (p)-[:HAS_INVENTORY_INFO]->(inv);

MATCH (p:Product {sku: 'SKU2'}), (s:Sale {number_of_products_sold: 8}), (inv:Inventory {stock_levels: 1})
CREATE (p)-[:HAS_SALE_INFO]->(s)
CREATE (p)-[:HAS_INVENTORY_INFO]->(inv);

// Conectar Productos con su información de Producción y Calidad
MATCH (p:Product {sku: 'SKU0'}), (prod:Production {production_volumes: 215}), (qc:QualityCheck {inspection_results: 'Pending'})
CREATE (p)-[:HAS_PRODUCTION_INFO]->(prod)
CREATE (p)-[:UNDERGOES_QUALITY_CHECK]->(qc);

MATCH (p:Product {sku: 'SKU1'}), (prod:Production {production_volumes: 517}), (qc:QualityCheck {inspection_results: 'Fail'})
CREATE (p)-[:HAS_PRODUCTION_INFO]->(prod)
CREATE (p)-[:UNDERGOES_QUALITY_CHECK]->(qc);

MATCH (p:Product {sku: 'SKU2'}), (prod:Production {production_volumes: 971}), (qc:QualityCheck {inspection_results: 'Pass'})
CREATE (p)-[:HAS_PRODUCTION_INFO]->(prod)
CREATE (p)-[:UNDERGOES_QUALITY_CHECK]->(qc);

// Conectar Productos con sus Proveedores y Ubicaciones
MATCH (p:Product {sku: 'SKU0'}), (sup:Supplier {name: 'Supplier 3'}), (loc:Location {city: 'Mumbai'})
CREATE (p)<-[:SUPPLIES]-(sup)
CREATE (sup)-[:LOCATED_IN]->(loc);

MATCH (p:Product {sku: 'SKU1'}), (sup:Supplier {name: 'Supplier 1'}), (loc:Location {city: 'Kolkata'})
CREATE (p)<-[:SUPPLIES]-(sup)
CREATE (sup)-[:LOCATED_IN]->(loc);

MATCH (p:Product {sku: 'SKU2'}), (sup:Supplier {name: 'Supplier 5'}), (loc:Location {city: 'Delhi'})
CREATE (p)<-[:SUPPLIES]-(sup)
CREATE (sup)-[:LOCATED_IN]->(loc);

// Conectar la información de Envío con Transportistas, Rutas y Modos de Transporte
MATCH (ship:Shipment {order_quantities: 96}), (sc:ShippingCarrier {name: 'Carrier B'}), (r:Route {name: 'Route B'}), (tm:TransportationMode {mode: 'Road'})
CREATE (ship)-[:HANDLED_BY]->(sc)
CREATE (ship)-[:USES_ROUTE]->(r)
CREATE (ship)-[:USES_TRANSPORTATION_MODE]->(tm);

MATCH (ship:Shipment {order_quantities: 37}), (sc:ShippingCarrier {name: 'Carrier A'}), (r:Route {name: 'Route C'}), (tm:TransportationMode {mode: 'Air'})
CREATE (ship)-[:HANDLED_BY]->(sc)
CREATE (ship)-[:USES_ROUTE]->(r)
CREATE (ship)-[:USES_TRANSPORTATION_MODE]->(tm);

MATCH (ship:Shipment {order_quantities: 88}), (sc:ShippingCarrier {name: 'Carrier C'}), (r:Route {name: 'Route A'}), (tm:TransportationMode {mode: 'Rail'})
CREATE (ship)-[:HANDLED_BY]->(sc)
CREATE (ship)-[:USES_ROUTE]->(r)
CREATE (ship)-[:USES_TRANSPORTATION_MODE]->(tm);


"""